In [9]:
import numpy as np
import pandas as pd

# 垂直


## 垂直相减 diff


In [11]:
df = pd.DataFrame({"a": [10, 20, 15, 50, 40]})  
df

,a
0,10
1,20
2,15
3,50
4,40


In [13]:
df["a"].diff(1) # 上面减下面

0     NaN
1    10.0
2    -5.0
3    35.0
4   -10.0
Name: a, dtype: float64

In [14]:
df["a"].diff(-1) # 下面减上面

0   -10.0
1     5.0
2   -35.0
3    10.0
4     NaN
Name: a, dtype: float64

## 垂直减+除 pct_change


In [15]:
df["a"].pct_change(1)  #减完了之后，再除


0         NaN
1    1.000000
2   -0.250000
3    2.333333
4   -0.200000
Name: a, dtype: float64

In [16]:
df["a"].pct_change(-1)

0   -0.500000
1    0.333333
2   -0.700000
3    0.250000
4         NaN
Name: a, dtype: float64

# 滚动 rolling


In [2]:
df = pd.DataFrame(
    {"col1": list(range(7)), "col2": list(range(1, 8)), "col3": list(range(3, 10))}
)
df

,col1,col2,col3
0,0,1,3
1,1,2,4
2,2,3,5
3,3,4,6
4,4,5,7
5,5,6,8
6,6,7,9


## type of rolling


In [3]:
type(df.rolling(3))

pandas.core.window.rolling.Rolling

## 滚动计算 sum/agg/mean


In [4]:
df.rolling(3).sum()

,col1,col2,col3
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,3.0,6.0,12.0
3,6.0,9.0,15.0
4,9.0,12.0,18.0
5,12.0,15.0,21.0
6,15.0,18.0,24.0


In [5]:
df.rolling(3).agg(lambda x: sum(x))

,col1,col2,col3
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,3.0,6.0,12.0
3,6.0,9.0,15.0
4,9.0,12.0,18.0
5,12.0,15.0,21.0
6,15.0,18.0,24.0


In [6]:
# 传将不同的函数应用在不同的列
df.rolling(3).agg({"col1": "sum", "col2": "mean", "col3": "min"})

,col1,col2,col3
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,3.0,2.0,3.0
3,6.0,3.0,4.0
4,9.0,4.0,5.0
5,12.0,5.0,6.0
6,15.0,6.0,7.0


## 自定义计算 apply


In [11]:
def func(param):
    print(param)
    print(type(param))
    return param.sum() + param.min()


df.rolling(4).apply(func)

0    0.0
1    1.0
2    2.0
3    3.0
dtype: float64
<class 'pandas.core.series.Series'>
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64
<class 'pandas.core.series.Series'>
2    2.0
3    3.0
4    4.0
5    5.0
dtype: float64
<class 'pandas.core.series.Series'>
3    3.0
4    4.0
5    5.0
6    6.0
dtype: float64
<class 'pandas.core.series.Series'>
0    1.0
1    2.0
2    3.0
3    4.0
dtype: float64
<class 'pandas.core.series.Series'>
1    2.0
2    3.0
3    4.0
4    5.0
dtype: float64
<class 'pandas.core.series.Series'>
2    3.0
3    4.0
4    5.0
5    6.0
dtype: float64
<class 'pandas.core.series.Series'>
3    4.0
4    5.0
5    6.0
6    7.0
dtype: float64
<class 'pandas.core.series.Series'>
0    3.0
1    4.0
2    5.0
3    6.0
dtype: float64
<class 'pandas.core.series.Series'>
1    4.0
2    5.0
3    6.0
4    7.0
dtype: float64
<class 'pandas.core.series.Series'>
2    5.0
3    6.0
4    7.0
5    8.0
dtype: float64
<class 'pandas.core.series.Series'>
3    6.0
4    7.0
5    8.0
6    9.0
dtype: 

,col1,col2,col3
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,6.0,11.0,21.0
4,11.0,16.0,26.0
5,16.0,21.0,31.0
6,21.0,26.0,36.0


## 跳跃计算 step


In [8]:
# step=2 把rolling的计算结果分步切片
# sum的结果和rolling一致，单纯但少了一些数据，少了的这些数据不影响计算
rolling_step = df.rolling(3, step=2).sum()
rolling_step

,col1,col2,col3
0,NaN,NaN,NaN
2,3.0,6.0,12.0
4,9.0,12.0,18.0
6,15.0,18.0,24.0


## max/min


In [10]:
data = {"open": np.random.rand(7), "close": np.random.rand(7)}
df_max_min = pd.DataFrame(data)

# 使用 rolling 计算前 4 根 open 和 close 的最大值
rolling_max_open = df_max_min["open"].shift(1).rolling(window=4).max()
rolling_max_close = df_max_min["close"].shift(1).rolling(window=4).max()
rolling_min_open = df_max_min["open"].shift(1).rolling(window=4).min()
rolling_min_close = df_max_min["close"].shift(1).rolling(window=4).min()

# 合并结果为一个新的列 max_price
df_max_min["max_price"] = pd.concat([rolling_max_open, rolling_max_close], axis=1).max(
    axis=1
)
df_max_min["min_price"] = pd.concat([rolling_min_open, rolling_min_close], axis=1).min(
    axis=1
)
df_max_min

,open,close,max_price,min_price
0,0.736058,0.592916,NaN,NaN
1,0.723240,0.022988,NaN,NaN
2,0.733641,0.044763,NaN,NaN
3,0.670085,0.004990,NaN,NaN
4,0.773411,0.591445,0.736058,0.00499
5,0.859118,0.185387,0.773411,0.00499
6,0.249464,0.818621,0.859118,0.00499


# 偏移 shift

把数据整体向 前/后 偏移


In [3]:
df_value = pd.DataFrame({"values": [1, 2, 3, 4, 5]})

# 将数据向前移动一个位置
df_value["shifted_values"] = df_value["values"].shift(1)
df_value

,values,shifted_values
0,1,NaN
1,2,1.0
2,3,2.0
3,4,3.0
4,5,4.0


# 累计 cum


In [4]:
df = pd.DataFrame({"a": [10, 20, 15, 50, 40]})

## 累计最大值


In [5]:
print(df["a"].cummax())

0    10
1    20
2    20
3    50
4    50
Name: a, dtype: int64


## 累计最小值


In [6]:
print(df["a"].cummin())

0    10
1    10
2    10
3    10
4    10
Name: a, dtype: int64


## 累乘


In [7]:
print(df["a"].cumprod())

0         10
1        200
2       3000
3     150000
4    6000000
Name: a, dtype: int64


## 累和


In [8]:
print(df["a"].cumsum())

0     10
1     30
2     45
3     95
4    135
Name: a, dtype: int64
